In [1]:
import itertools
import json
import os
import numpy as np
import tensorflow as tf
import keras
from keras.models import Model
from keras.models import load_model
from keras.layers import Dense, Dropout, Flatten, Reshape, Input
from keras.layers import Conv2D, MaxPooling2D, LeakyReLU, Activation
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD
from keras.regularizers import l2

from lib.config import BOARD_SIZE

/home/nuwanda/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
L2_E = 1e-4

def get_model():
    model_input = Input(shape=(BOARD_SIZE, BOARD_SIZE))
    input_shape = Reshape((BOARD_SIZE, BOARD_SIZE, 1))(model_input)
    init_conv = Activation('relu')(Conv2D(128, (5, 5), padding='same')(input_shape))

    main_conv1 = Activation('relu')(Conv2D(128, (3, 3), padding='same', kernel_regularizer=l2(L2_E))(init_conv))
    main_conv2 = Activation('relu')(Conv2D(128, (3, 3), padding='same', kernel_regularizer=l2(L2_E))(main_conv1))
    main_conv3 = Activation('relu')(Conv2D(128, (3, 3), padding='same', kernel_regularizer=l2(L2_E))(main_conv2))
    main_conv4 = Activation('relu')(Conv2D(128, (3, 3), padding='same', kernel_regularizer=l2(L2_E))(main_conv3))

    conv_probs = Activation('relu')(Conv2D(4, (1, 1), activation="relu", kernel_regularizer=l2(L2_E), padding='same')(main_conv4))
    flat_probs = Flatten()(conv_probs)
    probs = Dense(BOARD_SIZE*BOARD_SIZE, activation="softmax", kernel_regularizer=l2(L2_E), name='probs')(flat_probs)
    
    conv_win = Activation('relu')(Conv2D(2, (1, 1), padding='same', kernel_regularizer=l2(L2_E))(main_conv4))
    flat_win = Flatten()(conv_win)
    win = Dense(1, activation='tanh', name='win', kernel_regularizer=l2(L2_E))(flat_win)

    model = Model(inputs=model_input, outputs=[probs, win])
    model.compile(
        loss=['categorical_crossentropy', 'mean_squared_error'], optimizer='adam', metrics=['accuracy']
    )
    return model

In [3]:
def generator(data):
    size, X, y, win = 0, [], [], []
    for item in data:
        X.append(item[0])
        y.append(item[1])
        win.append(item[2])
        size += 1
        if size == BATCH_SIZE:
            yield np.float64(np.array(X)).reshape((size, 15, 15)), \
                  [np.array(y).reshape((size, 225)), np.array(win).reshape((size, 1))]
            size, X, y, win = 0, [], [], []

In [4]:
def get_additional_train_data():
    pass


In [5]:
def get_train_data(ind=0):
    with open('good_ones/policy_{}.json'.format(str(ind))) as f:
        train_data = json.load(f)
    return train_data

In [6]:
from random import shuffle
model = get_model()
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 15, 15)       0                                            
__________________________________________________________________________________________________
reshape_1 (Reshape)             (None, 15, 15, 1)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 15, 15, 128)  3328        reshape_1[0][0]                  
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 15, 15, 128)  0           conv2d_1[0][0]                   
_____________________________________

In [ ]:
PATH_TO_MODELS = 'good_ones/policy_{}.h5'
MODEL = None
BATCH_SIZE = 32

for ind in range(0, 400):
    if ind < 360:
        continue
    if os.path.isfile(PATH_TO_MODELS.format(str(ind))):
        if not os.path.isfile(PATH_TO_MODELS.format(str(ind + 1))):
            print('model loaded from file ' + PATH_TO_MODELS.format(str(ind)))
            model = load_model(PATH_TO_MODELS.format(str(ind)))
        continue
    train_data = get_train_data(ind)
    print('index {}, data size {}'.format(str(ind), len(train_data)))
    model.fit_generator(generator(train_data), epochs=1, verbose=1,
                        steps_per_epoch=len(train_data) // BATCH_SIZE,
                        shuffle=True)
    model.save(PATH_TO_MODELS.format(str(ind)))
    Model = model

model loaded from file renju_nn_big_model/policy_306.h5
index 308, data size 129616
Epoch 1/1
4050/4050 [==============================] - 243s 60ms/step - loss: 2.9464 - probs_loss: 1.7774 - win_loss: 0.9704 - probs_acc: 0.4935 - win_acc: 0.0296
index 309, data size 143532
Epoch 1/1
4485/4485 [==============================] - 271s 60ms/step - loss: 2.8918 - probs_loss: 1.7205 - win_loss: 0.9687 - probs_acc: 0.5271 - win_acc: 0.0314
index 310, data size 146268
Epoch 1/1
4570/4570 [==============================] - 276s 60ms/step - loss: 3.0939 - probs_loss: 1.9223 - win_loss: 0.9684 - probs_acc: 0.4688 - win_acc: 0.0316
index 311, data size 133344
Epoch 1/1
4167/4167 [==============================] - 234s 56ms/step - loss: 3.0930 - probs_loss: 1.9179 - win_loss: 0.9734 - probs_acc: 0.4619 - win_acc: 0.0266
index 312, data size 125300
Epoch 1/1
3915/3915 [==============================] - 236s 60ms/step - loss: 3.0230 - probs_loss: 1.8478 - win_loss: 0.9741 - probs_acc: 0.4770 - win_a